# 02 : GPU Check

This is a simple test to see if the GPU is available and working correctly.

- https://stackoverflow.com/questions/76581229/is-it-possible-to-check-if-gpu-is-available-without-using-deep-learning-packages
- https://docs.mlrun.org/en/v1.7.2/runtimes/configuring-job-resources.html
- https://docs.k3s.io/advanced#nvidia-container-runtime

In [1]:
import mlrun

In [2]:
# Show the API server URL
mlrun.get_run_db()

HTTPRunDB('http://dragon.local:30070')

In [3]:
# Set the base project name
project_name = "mlrun-demo"

# Initialize the MLRun project object
project = mlrun.get_or_create_project(
    name=project_name, 
    context="./",
    user_project=True)

# Display the current project name
project_name = project.metadata.name
print(f'Full project name: {project_name}')

> 2025-07-31 00:20:45,890 [info] Project loaded successfully: {"project_name":"mlrun-demo-johannes"}
Full project name: mlrun-demo-johannes


## Get GPU Function

In [4]:
%%writefile 02_get_gpu_info.py

import GPUtil
import subprocess

def get_gpu_info(context):    
    gpus = GPUtil.getGPUs()
    gpu_info = []
    for gpu in gpus:
        gpu_info.append({
            'id': gpu.id,
            'name': gpu.name,
            'load': gpu.load,
            'memory_total': gpu.memoryTotal,
            'memory_free': gpu.memoryFree,
            'memory_used': gpu.memoryUsed,
        })

    print(f"GPU Info v4: {gpu_info}")
    context.logger.info(f"GPU Info: {gpu_info}")

    # execute the nvidia-smi command on the cli to get detailed GPU info
    try:
        nvidia_smi_output = subprocess.check_output(['nvidia-smi'], universal_newlines=True)
        print("NVIDIA-SMI Output:")
        print(nvidia_smi_output)
        context.logger.info(f"NVIDIA-SMI Output:\n{nvidia_smi_output}")
    except Exception as e:
        error_msg = f"Error running nvidia-smi: {str(e)}"
        print(error_msg)
        context.logger.warning(error_msg)
    
    return gpu_info

Overwriting 02_get_gpu_info.py


## ML Run Function

In [7]:
#image = "mlrun/mlrun-gpu:1.9.1-py39"
#image =  "dragon:30500/mlrun/mlrun-gpu:1.9.1-py39"
#image =  "mlrun/mlrun-gpu:1.7.2"
image =  "mlrun/mlrun:1.7.2"

fn_gpu_check = project.set_function(
    func="02_get_gpu_info.py",
    name="gpu-check",
    tag="latest",
    kind="job",
    image=image,
    handler="get_gpu_info",
    requirements=["GPUtil==1.4.0"])

# Then set the GPU resources on the function's spec
fn_gpu_check.with_limits(mem="2G", cpu=2, gpus=1)  # upper bound

# build the function
project.build_function(function='gpu-check')

> 2025-07-31 00:23:16,106 [warning] It is recommended to use k8s secret (specify secret_name), specifying aws_access_key/aws_secret_key directly is unsafe.


The `overwrite_build_params` parameter default will change from 'False' to 'True' in 1.10.0.


> 2025-07-31 00:23:16,316 [info] Started building image: .mlrun/func-mlrun-demo-johannes-gpu-check:latest
INFO[0004] Retrieving image manifest mlrun/mlrun:1.7.2  
INFO[0004] Retrieving image mlrun/mlrun:1.7.2 from registry index.docker.io 
INFO[0026] Built cross stage deps: map[]                
INFO[0026] Retrieving image manifest mlrun/mlrun:1.7.2  
INFO[0026] Returning cached image manifest              
INFO[0026] Executing 0 build triggers                   
INFO[0026] Building stage 'mlrun/mlrun:1.7.2' [idx: '0', base-idx: '-1'] 
INFO[0026] Unpacking rootfs as cmd RUN echo 'Installing /empty/requirements.txt...'; cat /empty/requirements.txt requires it. 
INFO[0421] RUN echo 'Installing /empty/requirements.txt...'; cat /empty/requirements.txt 
INFO[0421] Initializing snapshotter ...                 
INFO[0421] Taking snapshot of full filesystem...        
INFO[0436] Cmd: /bin/sh                                 
INFO[0436] Args: [-c echo 'Installing /empty/requirements.txt...'; cat

BuildStatus(ready=True, outputs={'image': '.mlrun/func-mlrun-demo-johannes-gpu-check:latest'})

In [ ]:
# run the function locally
fn_gpu_check.run(
    local=False,
    handler="get_gpu_info",
    auto_build=True
)

> 2025-07-31 00:40:30,323 [info] Storing function: {"db":"http://dragon.local:30070","name":"gpu-check-get-gpu-info","uid":"8d72aae5322747369d2c68ce9df9e70a"}
> 2025-07-31 00:40:30,539 [info] Job is running in the background, pod: gpu-check-get-gpu-info-7x2pn


In [ ]:
# Refresh the function from the saved .py file (this is instant)
project.set_function(func="02_get_gpu_info.py", name="gpu-check")

# run the function again [you can modify the py file and set the function again = this will not rebuild the entire thing] 
project.run_function(
    function='gpu-check',
    handler='get_gpu_info'
)